<a href="https://colab.research.google.com/github/NPCA-TEAM/COVID-19/blob/main/Scripts/%209%20-%20ANALISE_POS_PROJE%C3%87%C3%83O_covid_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação preliminar

In [ ]:
# concede permissão de acesso ao drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dependências

In [ ]:
!pip install darts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 592.0/592.0 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4

In [ ]:
!pip install matplotlib==3.1.3

In [ ]:
!pip install pyyaml==5.4.1

## Bibliotecas

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import glob
import os

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime
from darts import timeseries

from darts.metrics import mape, mse, rmse, coefficient_of_variation

# Classe principal

In [ ]:
class Avaliacao:

  def __init__(self, sintoma_or_publicacao_or_ocorrencia, casos_obitos, modelo):
        self.casos_obitos = casos_obitos
        self.sintoma_or_publicacao_or_ocorrencia = sintoma_or_publicacao_or_ocorrencia
        self.modelo = modelo
        self.forecast_date = '28_11_2022' # Alterar para a data do arquivo mais recente
        self.current_date = '04_12_2022' # Alterar para a data da analise

        self.main_path = f'/content/drive/MyDrive/NPCA - COVID/{self.casos_or_obitos_folder}'

# ========================================================================================
# ------------------------------------- Setters ------------------------------------------
# ========================================================================================


  @property
  def casos_or_obitos_folder(self):
    if self.casos_obitos == 'casos':
      self.casos_or_obitos = '_CASOS'
    elif self.casos_obitos == 'obitos':
      self.casos_or_obitos = '_OBITOS'
    return self.casos_or_obitos


  @property
  def sintoma_or_publicacao_or_ocorrencia_column(self):
    if self.casos_obitos == 'casos':
      if self.sintoma_or_publicacao_or_ocorrencia == 'sintomas': # Pegando os dados de sintoma ou publicação
        column = 'DataSintoma'
      elif self.sintoma_or_publicacao_or_ocorrencia == 'publicacao':
        column = 'Publicacao'
    elif self.casos_obitos == 'obitos': 
      if self.sintoma_or_publicacao_or_ocorrencia == 'ocorrencia': # Pegando os dados de sintoma ou publicação
        column = 'DataOcorrencia'
      elif self.sintoma_or_publicacao_or_ocorrencia == 'publicacao':
        column = 'Publicacao'
    return column


  @property
  def casos_obitos_column(self):
    if self.casos_obitos == 'casos':
      first = 'Casos'
    elif self.casos_obitos == 'obitos':
      first = 'Obitos'
    return first


  @property
  def real_model(self): # Extraindo os dados reais
    
    # Carregar o dataframe
    path_dataset = f'{self.main_path}/DataSet/'  
    file_list = glob.glob(path_dataset + '*')
    file_path = list(filter(lambda x: self.current_date in x, file_list))[0]
    dataframe = pd.read_excel(file_path)
    
    # Pós Processamento
    # Valores < 0 = 0
    dataframe = dataframe.loc[:, ['data', f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA']]
    dataframe[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA'] = np.where(dataframe[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA'] <= 0, 0.00001, dataframe[f'{self.casos_obitos_column}_{self.sintoma_or_publicacao_or_ocorrencia_column}_MM_atual_PA'])
    
    t_s = timeseries.TimeSeries.from_dataframe(df=dataframe, time_col= 'data')
    return t_s


  @property
  def read_and_process_models(self): # Leitura e processamento dos conjuntos de dados
   
    # Carregar o dataframe
    path = f'{self.main_path}/FittedForecastsPosSelectedModel/{self.casos_obitos}_{self.sintoma_or_publicacao_or_ocorrencia}_fittedforecast_{self.modelo}_{self.getForecastDate}.csv'
    df = pd.read_csv(path)

    # Pós Processamento
    # Valores < 0 = 0
    valForecasts = df.loc[:, ['Date', 'Simulated']]
    valForecasts['Simulated'] = np.where(valForecasts['Simulated'] <= 0, 0.00001, valForecasts['Simulated'])

    series_forecast = timeseries.TimeSeries.from_dataframe(df=valForecasts,  time_col= 'Date')
    return series_forecast


# ========================================================================================
# ----------------------------------- Getters --------------------------------------------
# ========================================================================================


  @property
  def getForecastDate(self): # Get data dos dados (mesma para reais e forecast)
    return self.forecast_date

  @property
  def getCurrentDate(self): # Get data dos dados (mesma para reais e forecast)
    return self.current_date

# ========================================================================================
# ----------------------------------- Tabela de erros ------------------------------------
# ========================================================================================


  def TableMetrics(self):
    realModel = self.real_model
    readAndProcessModels = self.read_and_process_models
    values = []

    metric_mape = mape(realModel, readAndProcessModels) 
    metric_mse = mse(realModel, readAndProcessModels)
    metric_rmse = rmse(realModel, readAndProcessModels)
    metric_coefficient_of_variation = coefficient_of_variation(realModel, readAndProcessModels)

    values.append([f'{self.casos_obitos} {self.sintoma_or_publicacao_or_ocorrencia}', self.modelo, metric_mape, metric_mse, metric_rmse, metric_coefficient_of_variation, self.getCurrentDate]) 

    df = pd.DataFrame(columns = ['AVALIADO', 'MODELO', 'MAPE', 'MSE', 'RMSE', 'COEFFICIENT OF VARIATION', 'DATA']) # Criando df vazio para anexar notas

    for i in values:
      result = dict(zip(['AVALIADO', 'MODELO', 'MAPE', 'MSE', 'RMSE', 'COEFFICIENT OF VARIATION', 'DATA'], i)) # Criando dicionário com resultados
      df_to_append = pd.DataFrame(result, index = ['1']) # Criando DF com resultados
      df = pd.concat([df, df_to_append])

    return df
  

  def saveMetricsTable(self): # Salvando tabela de erros
    save_path = f'{self.main_path}/PostProjectionAnalysis/{self.casos_obitos}_{self.sintoma_or_publicacao_or_ocorrencia}_analysis_{self.modelo}_{self.getCurrentDate}.csv'
    self.TableMetrics().to_csv(save_path, index=False)


# ========================================================================================
# ------------------------------- Gráficos de comparação ---------------------------------
# ========================================================================================

  def getTitulo(self, ultimos30):

    if self.casos_obitos == 'casos':
      CasosOrObitos = 'cases'
    elif self.casos_obitos == 'obitos':
      CasosOrObitos = 'deaths'

    if self.sintoma_or_publicacao_or_ocorrencia == 'publicacao':
      SintoOrPubliOrOcorr = 'publication'
    elif self.sintoma_or_publicacao_or_ocorrencia == 'sintomas':
      SintoOrPubliOrOcorr = 'the first symptoms'
    elif self.sintoma_or_publicacao_or_ocorrencia == 'ocorrencia':
      SintoOrPubliOrOcorr = 'occurrence'
    
    # Criação do titulo do grafico.
    if ultimos30 is False:
      titulo = f'Forecast of {CasosOrObitos} by day of {SintoOrPubliOrOcorr} (Analysis - moving average) - {self.modelo}'
    elif ultimos30 is True:
      titulo = f'Forecast of {CasosOrObitos} by day of {SintoOrPubliOrOcorr} (Analysis - moving average) - Last 30 days - {self.modelo}'

    return titulo


  def plot(self, ultimos30=False):

    median_plot = self.read_and_process_models.pd_dataframe()
    real_plot = self.real_model.pd_dataframe()
    titulo = self.getTitulo(ultimos30)
    inter = 20

    if ultimos30 is True:
      median_plot = median_plot.tail(30)
      real_plot = real_plot.tail(30)
      inter = 1
    
    plt.style.use('seaborn-dark-palette')
    fig, ax = plt.subplots(figsize=(18,6))

    ax.plot(real_plot, lw=1.5, linestyle='solid', label='Observed') # Dados observados atuais
    ax.plot(median_plot[:-6], lw=1.5, linestyle='solid', label='Historical forecast') # Historical forecast de 7 dias atras
    ax.plot(median_plot[-7:], lw=1.5, linestyle='solid', label='Ahead forecast') # Predição de 7 dias atras
    ax.axvline(x=median_plot.index[-7], alpha=.5, linestyle='--') #Linha vertical

    min, max = ax.get_ylim()
    ax.yaxis.set_ticks(np.arange(0, max, max/20))
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=inter)) # Intervalo entre as marcações de tempo
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d'))

    ax.legend(loc='upper left', ncol = 2, fancybox=True, edgecolor='Black', framealpha=1)
    ax.set(title = titulo) # Titulo :)
    ax.margins(0.015, tight=True)
    ax.grid(True)

    fig.autofmt_xdate(rotation=75)
    fig.tight_layout()

    fig.savefig(f'{self.main_path}/PostProjectionAnalysis/Plots/{titulo} - {self.getCurrentDate}.png', dpi=300)
  

  def getGraphs(self):
    self.plot(False)
    self.plot(True)


# ========================================================================================
# --------------------------------------- Geral ------------------------------------------
# ========================================================================================


  def analyse(self):
    display(self.TableMetrics())
    self.saveMetricsTable()
    self.getGraphs()



# Analise

In [ ]:
# É necessario mudar o nome do modelo manualmente
casos_sintoma = Avaliacao('sintomas', 'casos', 'NHITS')
casos_sintoma.analyse()

In [ ]:
# Mudar o nome do modelo
casos_publicacao = Avaliacao('publicacao', 'casos', 'TRANSFORMER')
casos_publicacao.analyse()

In [ ]:
# Não esquece de mudar o nome do modelo
obitos_ocorrencia = Avaliacao('ocorrencia', 'obitos', 'NBEATS')
obitos_ocorrencia.analyse()

In [ ]:
# Tem que mudar o nome do modelo aqui tb
obitos_publicacao = Avaliacao('publicacao', 'obitos', 'NBEATS')
obitos_publicacao.analyse()